In [19]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp


In [20]:
project_name = 'apache-log4j'
project_releases = pd.read_csv('complete_links_allVersions_limit20.csv')

In [21]:
project_releases.head()

,project_name,project_link,version_name,commit,major_version_x,major_version_int,major_version_y,count,rank
0,web3j-web3j,https://github.com/web3j/web3j,v4.6.1,"Commit(sha=""8e94d2bcf305358d030bab9f89d4692bb3...",4,4.0,4,30,1.0
1,web3j-web3j,https://github.com/web3j/web3j,v4.6.0-android,"Commit(sha=""9ff030a628caf2c3a77e8952bbf7feb890...",4,4.0,4,30,1.0
2,web3j-web3j,https://github.com/web3j/web3j,v4.6.0,"Commit(sha=""5877c308e11c0de8e24e50c882effa8435...",4,4.0,4,30,1.0
3,web3j-web3j,https://github.com/web3j/web3j,v4.5.9,"Commit(sha=""6c23f373ce8c30a38331c4eebf46517be5...",4,4.0,4,30,1.0
4,web3j-web3j,https://github.com/web3j/web3j,v4.5.8,"Commit(sha=""62df9d1c91281664c8534b994a4bbf0941...",4,4.0,4,30,1.0


In [22]:
finished_project_arr = []
finished_project = open('finished_projects.txt','r')
for line in finished_project:
    finished_project_arr.append(line.split('\n')[0])
finished_project.close()
for project_name in project_releases['project_name'].unique():
    if project_name not in finished_project_arr:
        print(project_name)

web3j-web3j
voxeolabs-moho
uber-NullAway
testcontainers-testcontainers-java
swagger-api-swagger-core
square-retrofit
square-okhttp
square-android-times-square
springfox-springfox
spring-projects-spring-security
spring-projects-spring-boot
sofastack-sofa-rpc
seleniumHQ-selenium
scribejava-scribejava
rzwitserloot-lombok
robolectric-robolectric
rest-assured-rest-assured
redisson-redisson
realm-realm-java
real-logic-aeron
raphw-byte-buddy
quarkusio-quarkus
pxb1988-dex2jar
pmd-pmd
permissions-dispatcher-PermissionsDispatcher
oracle-spacewalk-java
oracle-opengrok
oracle-oci-java-sdk
openengsb-openengsb
oblac-jodd
netty-netty
naver-pinpoint
mybatis-mybatis-3
mrniko-netty-socketio
mockito-mockito
lettuce-io-lettuce-core
knowm-XChange
kijiproject-kiji-bento
joelittlejohn-jsonschema2pojo
jhy-jsoup
jenkinsci-jenkins
jboss-switchyard-release
javaparser-javaparser
javafunk-funk
java-native-accessjna
jankotek-mapdb
izhangzhihao-intellij-rainbow-brackets
ionic-team-capacitor
immutables-immutables
iSo

In [23]:
current_project = pd.DataFrame(project_releases[project_releases['project_name'] == project_name])
current_project['rank'] = current_project['version_name'].rank()
current_project = current_project.sort_values(by=['rank'], ascending=False)
current_project.head(30)

,project_name,project_link,version_name,commit,major_version_x,major_version_int,major_version_y,count,rank
2940,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.9,"Commit(sha=""6ad133837a4c4f8199d00a05c3c16267db...",2,2.0,2,29,20.0
2941,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.8,"Commit(sha=""e97d3a2f65f09ef19e53c5431cc364c9fc...",2,2.0,2,29,19.0
2942,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.7,"Commit(sha=""e705d60f83a20366aa50407485f55e9c4b...",2,2.0,2,29,18.0
2943,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.6,"Commit(sha=""d7d1620197aa8dfd0e9462bcec15231567...",2,2.0,2,29,17.0
2944,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.5,"Commit(sha=""9839f89a0c35f24c21cc026ecc5b1664f7...",2,2.0,2,29,16.0
2945,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.4,"Commit(sha=""7a1faaf78c957ca7fefb61bffd74ce355f...",2,2.0,2,29,15.0
2946,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.3,"Commit(sha=""a2e141279d34c29090c80a84c6a47ec33a...",2,2.0,2,29,14.0
2947,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.2,"Commit(sha=""625c9048aa5ec2d38b850172fa9f0700c8...",2,2.0,2,29,13.0
2948,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9.1,"Commit(sha=""73fd550faa7b37ef31dd246752394cbb5f...",2,2.0,2,29,12.0
2949,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.9,"Commit(sha=""8f41ec9713568111cf6b81736d97b46aeb...",2,2.0,2,29,11.0


In [28]:
### Download dataset
try:
    os.mkdir('raw_sourcecode/' + project_name)
except:
    pass
for row in current_project.iterrows():
    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
    
    #print(command)
    os.system(command)
    
    command = 'git clone ' + row[1]['project_link'] +  ' C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']
    
    print(command)
    os.system(command)
    commit = row[1]['commit'].replace('Commit(sha="','')
    commit = commit.replace('")','')
    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name +'_' + row[1]['version_name'] + ' & git checkout ' + commit
    print(command)
    os.system(command)

git clone https://github.com/JodaOrg/joda-time C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.9
cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.9 & git checkout 6ad133837a4c4f8199d00a05c3c16267dbf6deb8
git clone https://github.com/JodaOrg/joda-time C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.8
cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.8 & git checkout e97d3a2f65f09ef19e53c5431cc364c9fcc3c8fb
git clone https://github.com/JodaOrg/joda-time C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.7
cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.7 & git checkout e705d60f83a20366aa50407485f55e9c4b15ff1b
git clone https://github.com/JodaOrg/joda-time

In [29]:
### Run Depends
try:
    os.mkdir('raw_depends/' + project_name)
except:
    pass
for row in current_project.iterrows():
    #command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
    #os.system(command)
    #print(command)
    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & ' + 'java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/'  + project_name + '/' + project_name + '_' + row[1]['version_name']+  ' ../raw_depends/' + project_name + "/" + project_name + '_' + row[1]['version_name'] 
    print(command)
    os.system(command)

cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.9 ../raw_depends/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.9
cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.8 ../raw_depends/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.8
cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.7 ../raw_depends/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.7
cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.6 ../raw_depends/JodaOrg-jod

In [113]:
try:
    os.mkdir('groundtruth/' + project_name)
except:
    pass

try:
    os.mkdir('MoJo_1.2.1/' + project_name)
except:
    pass


### Counter to only ensure the top 10 projects are selected
counter = 10

for row in current_project.iterrows():
    counter -= 1
    if counter < 0:
        break
    project_rank = row[1]['rank']
    version_name = row[1]['version_name']
    #print(project_rank)
    rootdir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']
    
    #print(rootdir)
    
    
    ### To obtain the current initial directory
    full_dir_arr = []
    for root, dirs, files in os.walk(rootdir):
        #print(root)
        #print(dirs)
        for element in files:
            if '.java' in element:
                dir_string = root + '\\' + element
                full_dir_arr.append(dir_string)

    cluster_dict = {}
    cluster_tree = {}


    for element in full_dir_arr:
        element = element.split('\\')
        child = element[-1]
        parent = element[-2]
        cluster_tree[child] = parent
            
    current_rank = project_rank
    while current_rank >= (project_rank - 9):
        current_rank -= 1
        project_to_be_compared = current_project[current_project['rank'] == current_rank]
        project_to_be_compared_rootdir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + project_to_be_compared['version_name'].values[0]
        #print(project_to_be_compared_rootdir)


        ### To obtain the current initial directory
        project_to_be_compared_full_dir_arr = []
        for root, dirs, files in os.walk(project_to_be_compared_rootdir):
            #print(root)
            #print(dirs)
            for element in files:
                if '.java' in element:
                    dir_string = root + '\\' + element
                    project_to_be_compared_full_dir_arr.append(dir_string)

        project_to_be_compared_cluster_dict = {}
        project_to_be_compared_cluster_tree = {}


        for element in project_to_be_compared_full_dir_arr:
            element = element.split('\\')
            child = element[-1]
            parent = element[-2]
            project_to_be_compared_cluster_tree[child] = parent
                
        cluster_tree =  {x:cluster_tree[x] for x in cluster_tree if x in project_to_be_compared_cluster_tree} 
    
    
        
    depends_dir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.json'
    #print(depends_dir)

    with open(depends_dir) as f:
        depends_results = json.load(f)
        index  = 0
        var_array = []
        for value in depends_results['variables']:
            var_array.append([index, value.split('\\')[-1]])
            #print(index, value)
            index += 1

        var_df = pd.DataFrame(var_array)
        var_df.columns = ['index_val', 'name']

        feature_list = {}
        feature_index = 2
        for element in depends_results['cells']:
            #print(element)
            try:
                for a in element['values']:
                    if a not in feature_list:
                        feature_list[a] = feature_index
                        feature_index += 1
                    #print(a['Call'])


            except:
                pass

        depends_dict = {}
        #print(len(depends_results['variables']))
        for i in range(len(depends_results['variables'])):
            key = depends_results['variables'][i].split('\\')[-1]
            depends_dict[key] = i

        arr_a_rsf = []
        filename = 'C:/Users/tanji/Desktop/SoftwareRemodularization/groundtruth/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.txt'
        with open(filename, 'w') as f:
            for key, value in cluster_tree.items():
                arr_a_rsf.append(depends_dict[key])
                f.write('contain ' + str(value).replace(" ",'') + ' ' + str(key).replace(' ','') + '\n')
            
        feature_arr = []
        
        for element in depends_results['cells']:
            #print(array)
            array = [0] * (len(feature_list) + 2)
            values = dict(element['values'])
            #print(element)
            array[0] = element['src']
            array[1] = element['dest']
            for feature in feature_list:
                try:
                    value = values[feature]
                    array[feature_list[feature]] = value
                except:
                    pass
            #print(array)
            feature_arr.append(array)

        feature_df = pd.DataFrame(feature_arr)
        col_names = ['src', 'dest']
        
        for element in feature_list:
            col_names.append(element)
        feature_df.columns = col_names
        try:
            os.mkdir('raw_mdg/' + project_name)
        except:
            pass
        
        try:
            os.mkdir('raw_mdg_results/' + project_name)
        except:
            pass
        test = feature_df[['src', 'dest']]
        
        tmp_mdg_filename = 'raw_mdg/' + project_name + '/' + project_name + '_' + version_name + '.mdg'
        #print(tmp_mdg_filename)
        tmp_mdg = open(tmp_mdg_filename, 'w+')
        for index, row in test.iterrows():
            #print(row['src'],row['dest'])
            tmp_mdg.write(str(row['src']) + ' ' + str(row['dest']) + '\n')

        tmp_mdg.close()
    
        bunch_result_filename = 'raw_mdg_results/' + project_name + '/' + project_name + '_' + version_name
    
        hillclimbing_filename = bunch_result_filename + '-hillclimbing'
        ga_filename = bunch_result_filename + '-ga'
        exhaustive_filename = bunch_result_filename + '-exhaustive'

        command = "java -jar alvin_bunch_automation.jar " + tmp_mdg_filename + " " + hillclimbing_filename +  " hillclimbing"
        print(command)
        os.system(command)

        command = "java -jar alvin_bunch_automation.jar " + tmp_mdg_filename + " " + ga_filename +  " ga"
        print(command)
        os.system(command)

        command = "java -jar alvin_bunch_automation.jar " + tmp_mdg_filename + " " + exhaustive_filename +  " exhaustive"
        print(command)
        os.system(command)
        
        
        
        ### This part is for hillclimbing. ###
        number = 0
        with open(hillclimbing_filename + '.bunch') as f:
            for line in f:
                number += 1
        
        filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'hillclimbing' + '_a.rsf'
        filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'hillclimbing' + '_b.rsf'

        #filename2 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + bunch_type + '_a' +".rsf"
        bunch_counter = 0
        max_element = 0
        arr_unique = []
        arr_b_rsf = []
        with open(filename_b, 'w') as f:
            len_b_rsf = 0
            bunch_file = open(hillclimbing_filename + '.bunch')
            table = []
            for line in bunch_file:
                bunch_counter += 1
                line = line.replace('\n', '')
                line = line.split(' = ')
                line[0] = line[0].replace('SS(', '')
                line[0] = line[0].replace(')', '')
                line[1] = line[1].split(sep = ', ')
                table.append(line)

            for i in range(len(table)):
                for j in range(len(table[i][1])):
                    #str(cluster_result[i])
                    string = "contain " + str(table[i][0]) + " " + str(table[i][1][j]) + "\n"
                    arr_unique.append(int(table[i][1][j]))
                    arr_b_rsf.append(int(table[i][1][j]))
                    if int(table[i][1][j]) > max_element:
                        max_element = int(table[i][1][j])
                    f.write(string)
                    len_b_rsf += 1
            bunch_file.close()

        f.close()
        
        with open(filename_a, 'w') as f:
            for key, value in cluster_tree.items():
                f.write('contain ' + str(value).replace(" ",'') + ' ' + str(depends_dict[key]).replace(' ','') + '\n')
        
        len_a_rsf = len(cluster_tree)
        
        to_be_added = list(set(arr_b_rsf) - set(arr_a_rsf))
        with open(filename_a,'a+') as f:
            for item in to_be_added:
                string = 'contain ' + str(item) + " " + str(item) + '\n'
                rsf_initial_counter += 1
                f.write(string)
        f.close()

        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
        with open(filename_b,'a+') as f:

            for item in to_be_added:
                string = 'contain ' + str(item) + " " + str(item) + '\n'
                rsf_initial_counter += 1
                f.write(string)
        f.close()
        
        command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
        #print(command)
        os.system(command)
            
        f.close()
        
        ### This part is for ga. ###
        number = 0
        with open(ga_filename + '.bunch') as f:
            for line in f:
                number += 1
        
        filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'ga' + '_a.rsf'
        filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'ga' + '_b.rsf'

        #filename2 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + bunch_type + '_a' +".rsf"
        bunch_counter = 0
        max_element = 0
        arr_unique = []
        arr_b_rsf = []
        with open(filename_b, 'w') as f:
            len_b_rsf = 0
            bunch_file = open(ga_filename + '.bunch')
            table = []
            for line in bunch_file:
                bunch_counter += 1
                line = line.replace('\n', '')
                line = line.split(' = ')
                line[0] = line[0].replace('SS(', '')
                line[0] = line[0].replace(')', '')
                line[1] = line[1].split(sep = ', ')
                table.append(line)

            for i in range(len(table)):
                for j in range(len(table[i][1])):
                    #str(cluster_result[i])
                    string = "contain " + str(table[i][0]) + " " + str(table[i][1][j]) + "\n"
                    arr_unique.append(int(table[i][1][j]))
                    arr_b_rsf.append(int(table[i][1][j]))
                    if int(table[i][1][j]) > max_element:
                        max_element = int(table[i][1][j])
                    f.write(string)
                    len_b_rsf += 1
            bunch_file.close()

        f.close()
        
        with open(filename_a, 'w') as f:
            for key, value in cluster_tree.items():
                f.write('contain ' + str(value).replace(" ",'') + ' ' + str(depends_dict[key]).replace(' ','') + '\n')
        
        len_a_rsf = len(cluster_tree)
        
        to_be_added = list(set(arr_b_rsf) - set(arr_a_rsf))
        with open(filename_a,'a+') as f:
            for item in to_be_added:
                string = 'contain ' + str(item) + " " + str(item) + '\n'
                rsf_initial_counter += 1
                f.write(string)
        f.close()

        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
        with open(filename_b,'a+') as f:

            for item in to_be_added:
                string = 'contain ' + str(item) + " " + str(item) + '\n'
                rsf_initial_counter += 1
                f.write(string)
        f.close()
        
        command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
        #print(command)
        os.system(command)
            
        f.close()
        
        ### This part is for exhaustive. ###
        number = 0
        with open(exhaustive_filename + '.bunch') as f:
            for line in f:
                number += 1
        
        filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'exhaustive' + '_a.rsf'
        filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + 'exhaustive' + '_b.rsf'

        #filename2 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + bunch_type + '_a' +".rsf"
        bunch_counter = 0
        max_element = 0
        arr_unique = []
        arr_b_rsf = []
        with open(filename_b, 'w') as f:
            len_b_rsf = 0
            bunch_file = open(exhaustive_filename + '.bunch')
            table = []
            for line in bunch_file:
                bunch_counter += 1
                line = line.replace('\n', '')
                line = line.split(' = ')
                line[0] = line[0].replace('SS(', '')
                line[0] = line[0].replace(')', '')
                line[1] = line[1].split(sep = ', ')
                table.append(line)

            for i in range(len(table)):
                for j in range(len(table[i][1])):
                    #str(cluster_result[i])
                    string = "contain " + str(table[i][0]) + " " + str(table[i][1][j]) + "\n"
                    arr_unique.append(int(table[i][1][j]))
                    arr_b_rsf.append(int(table[i][1][j]))
                    if int(table[i][1][j]) > max_element:
                        max_element = int(table[i][1][j])
                    f.write(string)
                    len_b_rsf += 1
            bunch_file.close()

        f.close()
        
        with open(filename_a, 'w') as f:
            for key, value in cluster_tree.items():
                f.write('contain ' + str(value).replace(" ",'') + ' ' + str(depends_dict[key]).replace(' ','') + '\n')
        
        len_a_rsf = len(cluster_tree)
        
        to_be_added = list(set(arr_b_rsf) - set(arr_a_rsf))
        with open(filename_a,'a+') as f:
            for item in to_be_added:
                string = 'contain ' + str(item) + " " + str(item) + '\n'
                rsf_initial_counter += 1
                f.write(string)
        f.close()

        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
        with open(filename_b,'a+') as f:

            for item in to_be_added:
                string = 'contain ' + str(item) + " " + str(item) + '\n'
                rsf_initial_counter += 1
                f.write(string)
        f.close()
        
        command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
        #print(command)
        os.system(command)
            
        f.close()
    

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/JodaOrg-joda-time/JodaOrg-joda-time_v2.9.9.json'

In [17]:
finished_project = open('finished_projects.txt','a+')
finished_project.write(project_name + '\n')
finished_project.close()

In [11]:
"""
counter = 10
for row in current_project.iterrows():
    counter -= 1
    if counter < 0:
        break
    project_rank = row[1]['rank']
    #print(project_rank)
    depends_dir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.json'
    #print(depends_dir)

    with open(depends_dir) as f:
        depends_results = json.load(f)
        index  = 0
        var_array = []
        for value in depends_results['variables']:
            var_array.append([index, value.split('\\')[-1]])
            #print(index, value)
            index += 1

        var_df = pd.DataFrame(var_array)
        var_df.columns = ['index_val', 'name']

        feature_list = {}
        feature_index = 2
        for element in depends_results['cells']:
            #print(element)
            try:
                for a in element['values']:
                    if a not in feature_list:
                        feature_list[a] = feature_index
                        feature_index += 1
                    #print(a['Call'])


            except:
                pass

        feature_arr = []
        for element in depends_results['cells']:
            #print(array)
            array = [0] * (len(feature_list) + 2)
            values = dict(element['values'])
            #print(element)
            array[0] = element['src']
            array[1] = element['dest']
            for feature in feature_list:
                try:
                    value = values[feature]
                    array[feature_list[feature]] = value
                except:
                    pass
            #print(array)
            feature_arr.append(array)

        feature_df = pd.DataFrame(feature_arr)
        col_names = ['src', 'dest']
        for element in feature_list:
            col_names.append(element)
        feature_df.columns = col_names

        feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
        G = nx.Graph()
        for index, row in feature_df.iterrows():
            G.add_edge(row['src'], row['dest'], weight=row['sum'])

        adj_mat = nx.adjacency_matrix(G)
        adj_mat_df = pd.DataFrame(adj_mat.todense())
        np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
        x = adj_mat_df.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        adj_mat_df = pd.DataFrame(x_scaled)
        
        
        n_cluster_divisible_arr = [5,6,7,8,9,10,11,12,13,14,15]
        affinity_arr = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
        linkage_arr = ['complete', 'average', 'single']
        
        for user_n_cluster in n_cluster_divisible_arr:
            for user_affinity in affinity_arr:
                for user_linkage in linkage_arr:
        
                    cluster = AgglomerativeClustering(n_clusters=user_n_cluster, affinity=user_affinity, linkage=user_linkage)
                    cluster_result = cluster.fit_predict(adj_mat_df)
                    
                    filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_a.rsf'
                    filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_b.rsf'
                    with open('test_b.rsf', 'w') as f:
                        len_b_rsf = 0
                        duplicate_array = []
                        for i in range(len(cluster_result)):
                            try:
                                subject = var_df[var_df['index_val'] == i]['name'].values[0]
                                #print(subject)
                                #print(subject in cluster_tree)
                                if subject in cluster_tree and subject not in duplicate_array:
                                    #print(element)
                                    duplicate_array.append(subject)
                                    string = "contain " + str(cluster_result[i]) + " " + subject + "\n"
                                    arr_b_rsf.append(subject)
                                    len_b_rsf += 1
                                    f.write(string)
                                else:
                                    print(subject)
                            except:
                                pass
                    f.close()

                    len_a_rsf = len(cluster_tree)

                    if len_b_rsf > len_a_rsf:
                        rsf_initial_counter = len_a_rsf
                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                        with open('test_a.rsf','a+') as f:

                            for item in to_be_added:
                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                rsf_initial_counter += 1
                                f.write(string)
                        f.close()

                    elif len_a_rsf > len_b_rsf:
                        print('here')
                        rsf_initial_counter = len_b_rsf
                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                        with open('test_b.rsf','a+') as f:

                            for item in to_be_added:
                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                rsf_initial_counter += 1
                                f.write(string)
                        f.close()
        
        
        
        f.close()
        
        
        
        
"""    
    
    

'\ncounter = 10\nfor row in current_project.iterrows():\n    counter -= 1\n    if counter < 0:\n        break\n    project_rank = row[1][\'rank\']\n    #print(project_rank)\n    depends_dir = \'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/\' + project_name + \'/\' + project_name + \'_\' + row[1][\'version_name\'] + \'.json\'\n    #print(depends_dir)\n\n    with open(depends_dir) as f:\n        depends_results = json.load(f)\n        index  = 0\n        var_array = []\n        for value in depends_results[\'variables\']:\n            var_array.append([index, value.split(\'\\\')[-1]])\n            #print(index, value)\n            index += 1\n\n        var_df = pd.DataFrame(var_array)\n        var_df.columns = [\'index_val\', \'name\']\n\n        feature_list = {}\n        feature_index = 2\n        for element in depends_results[\'cells\']:\n            #print(element)\n            try:\n                for a in element[\'values\']:\n                    if a not in featur

In [ ]:

index  = 0
var_array = []
for value in depends_results['variables']:
    var_array.append([index, value.split('\\')[-1]])
    #print(index, value)
    index += 1

var_df = pd.DataFrame(var_array)
var_df.columns = ['index_val', 'name']

feature_list = {}
feature_index = 2
for element in depends_results['cells']:
    #print(element)
    try:
        for a in element['values']:
            if a not in feature_list:
                feature_list[a] = feature_index
                feature_index += 1
            #print(a['Call'])


    except:
        pass

feature_arr = []
for element in depends_results['cells']:
    #print(array)
    array = [0] * (len(feature_list) + 2)
    values = dict(element['values'])
    #print(element)
    array[0] = element['src']
    array[1] = element['dest']
    for feature in feature_list:
        try:
            value = values[feature]
            array[feature_list[feature]] = value
        except:
            pass
    #print(array)
    feature_arr.append(array)

feature_df = pd.DataFrame(feature_arr)
col_names = ['src', 'dest']
for element in feature_list:
    col_names.append(element)
feature_df.columns = col_names

feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
G = nx.Graph()
for index, row in feature_df.iterrows():
    G.add_edge(row['src'], row['dest'], weight=row['sum'])

adj_mat = nx.adjacency_matrix(G)
adj_mat_df = pd.DataFrame(adj_mat.todense())
np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
x = adj_mat_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
adj_mat_df = pd.DataFrame(x_scaled)

In [ ]:
var_df.head()
#for row in var_df.iterrows():
    #print(row[1]['name'])

In [ ]:
print(len(var_df))

In [ ]:
cluster = AgglomerativeClustering(n_clusters=30, affinity='euclidean', linkage='complete')
cluster_result = cluster.fit_predict(adj_mat_df)

In [ ]:
print(len(cluster_result))
cluster_result

In [63]:
cluster_tree

{'AgeCalculator.java': 'time',
 'DateTimeBrowser.java': 'time',
 'DateTimePerformance.java': 'time',
 'Examples.java': 'time',
 'TimeZoneTable.java': 'time',
 'Chronology.java': 'time',
 'DateMidnight.java': 'time',
 'DateTime.java': 'time',
 'DateTimeComparator.java': 'time',
 'DateTimeConstants.java': 'time',
 'DateTimeField.java': 'time',
 'DateTimeFieldType.java': 'time',
 'DateTimeUtils.java': 'time',
 'DateTimeZone.java': 'time',
 'Days.java': 'time',
 'Duration.java': 'time',
 'DurationField.java': 'time',
 'DurationFieldType.java': 'time',
 'Hours.java': 'time',
 'IllegalFieldValueException.java': 'time',
 'IllegalInstantException.java': 'time',
 'Instant.java': 'time',
 'Interval.java': 'time',
 'JodaTimePermission.java': 'time',
 'LocalDate.java': 'time',
 'LocalDateTime.java': 'time',
 'LocalTime.java': 'time',
 'Minutes.java': 'time',
 'MonthDay.java': 'time',
 'Months.java': 'time',
 'MutableDateTime.java': 'time',
 'MutableInterval.java': 'time',
 'MutablePeriod.java': 't

In [68]:
to_be_added

[177]

In [103]:
arr_b_rsf

[166,
 167,
 328,
 290,
 47,
 236,
 164,
 327,
 60,
 196,
 197,
 206,
 79,
 195,
 186,
 288,
 102,
 191,
 100,
 182,
 295,
 284,
 280,
 292,
 223,
 172,
 319,
 156,
 158,
 160,
 188,
 23,
 170,
 144,
 145,
 312,
 311,
 147,
 28,
 215,
 63,
 216,
 165,
 163,
 154,
 155,
 148,
 302,
 130,
 153,
 146,
 152,
 54,
 149,
 253,
 254,
 259,
 251,
 123,
 16,
 17,
 298,
 112,
 293,
 97,
 242,
 246,
 116,
 110,
 291,
 66,
 78,
 181,
 240,
 175,
 109,
 114,
 39,
 323,
 324,
 326,
 80,
 71,
 325,
 161,
 0,
 13,
 48,
 36,
 37,
 199,
 135,
 19,
 289,
 106,
 299,
 297,
 281,
 282,
 285,
 67,
 138,
 220,
 209,
 183,
 95,
 279,
 273,
 180,
 91,
 94,
 278,
 272,
 276,
 277,
 275,
 264,
 274,
 263,
 171,
 40,
 300,
 128,
 261,
 174,
 92,
 72,
 260,
 241,
 255,
 257,
 258,
 6,
 117,
 2,
 256,
 81,
 82,
 88,
 189,
 252,
 113,
 294,
 98,
 108,
 96,
 20,
 1,
 3,
 322,
 59,
 213,
 53,
 248,
 50,
 239,
 143,
 308,
 310,
 4,
 141,
 321,
 159,
 250,
 52,
 317,
 320,
 157,
 151,
 316,
 99,
 101,
 287,
 38,
 115,
 

In [74]:
list(set(arr_a_rsf) - set(arr_b_rsf))

[177]

In [ ]:
with open('test_b.rsf', 'w') as f:
    len_b_rsf = 0
    duplicate_array = []
    for i in range(len(cluster_result)):
        try:
            subject = var_df[var_df['index_val'] == i]['name'].values[0]
            #print(subject)
            #print(subject in cluster_tree)
            if subject in cluster_tree and subject not in duplicate_array:
                #print(element)
                duplicate_array.append(subject)
                string = "contain " + str(cluster_result[i]) + " " + subject + "\n"
                arr_b_rsf.append(subject)
                len_b_rsf += 1
                f.write(string)
            else:
                print(subject)
        except:
            pass
    f.close()

len_a_rsf = len(cluster_tree)

if len_b_rsf > len_a_rsf:
    rsf_initial_counter = len_a_rsf
    to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
    with open('test_a.rsf','a+') as f:
        
        for item in to_be_added:
            string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
            rsf_initial_counter += 1
            f.write(string)
    f.close()
        
elif len_a_rsf > len_b_rsf:
    print('here')
    rsf_initial_counter = len_b_rsf
    to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
    with open('test_b.rsf','a+') as f:
        
        for item in to_be_added:
            string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
            rsf_initial_counter += 1
            f.write(string)
    f.close()

In [ ]:
'AppenderTable.java' in cluster_tree

In [ ]:
len(cluster_tree)

In [ ]:
arr_a_rsf = []
arr_b_rsf = []


In [46]:
cluster_tree
    
    


{'AgeCalculator.java': 'time',
 'DateTimeBrowser.java': 'time',
 'DateTimePerformance.java': 'time',
 'Examples.java': 'time',
 'TimeZoneTable.java': 'time',
 'Chronology.java': 'time',
 'DateMidnight.java': 'time',
 'DateTime.java': 'time',
 'DateTimeComparator.java': 'time',
 'DateTimeConstants.java': 'time',
 'DateTimeField.java': 'time',
 'DateTimeFieldType.java': 'time',
 'DateTimeUtils.java': 'time',
 'DateTimeZone.java': 'time',
 'Days.java': 'time',
 'Duration.java': 'time',
 'DurationField.java': 'time',
 'DurationFieldType.java': 'time',
 'Hours.java': 'time',
 'IllegalFieldValueException.java': 'time',
 'IllegalInstantException.java': 'time',
 'Instant.java': 'time',
 'Interval.java': 'time',
 'JodaTimePermission.java': 'time',
 'LocalDate.java': 'time',
 'LocalDateTime.java': 'time',
 'LocalTime.java': 'time',
 'Minutes.java': 'time',
 'MonthDay.java': 'time',
 'Months.java': 'time',
 'MutableDateTime.java': 'time',
 'MutableInterval.java': 'time',
 'MutablePeriod.java': 't

In [ ]:
list(set(test_arr_1) - set(test_arr_2))

In [70]:
arr_b_rsf

[166,
 167,
 328,
 290,
 47,
 236,
 164,
 327,
 60,
 196,
 197,
 206,
 79,
 195,
 186,
 288,
 102,
 191,
 100,
 182,
 295,
 284,
 280,
 292,
 223,
 172,
 319,
 156,
 158,
 160,
 188,
 23,
 170,
 144,
 145,
 312,
 311,
 147,
 28,
 215,
 63,
 216,
 165,
 163,
 154,
 155,
 148,
 302,
 130,
 153,
 146,
 152,
 54,
 149,
 253,
 254,
 259,
 251,
 123,
 16,
 17,
 298,
 112,
 293,
 97,
 242,
 246,
 116,
 110,
 291,
 66,
 78,
 181,
 240,
 175,
 109,
 114,
 39,
 323,
 324,
 326,
 80,
 71,
 325,
 161,
 0,
 13,
 48,
 36,
 37,
 199,
 135,
 19,
 289,
 106,
 299,
 297,
 281,
 282,
 285,
 67,
 138,
 220,
 209,
 183,
 95,
 279,
 273,
 180,
 91,
 94,
 278,
 272,
 276,
 277,
 275,
 264,
 274,
 263,
 171,
 40,
 300,
 128,
 261,
 174,
 92,
 72,
 260,
 241,
 255,
 257,
 258,
 6,
 117,
 2,
 256,
 81,
 82,
 88,
 189,
 252,
 113,
 294,
 98,
 108,
 96,
 20,
 1,
 3,
 322,
 59,
 213,
 53,
 248,
 50,
 239,
 143,
 308,
 310,
 4,
 141,
 321,
 159,
 250,
 52,
 317,
 320,
 157,
 151,
 316,
 99,
 101,
 287,
 38,
 115,
 

In [34]:
pwd

'C:\\Users\\tanji\\Desktop\\SoftwareRemodularization'

In [54]:
feature_arr

[[50, 63, 30.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0],
 [195, 17, 112.0, 0, 0, 2.0, 1.0, 13.0, 2.0, 4.0, 0, 0],
 [195, 16, 12.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [210, 57, 63.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [210, 53, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [291, 110, 3.0, 0, 0, 0, 0, 11.0, 0, 0, 0, 0],
 [217, 12, 2.0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0],
 [210, 59, 6.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [313, 142, 125.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [208, 141, 1.0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0],
 [217, 13, 4.0, 0, 0, 0, 0, 4.0, 0, 2.0, 0, 0],
 [208, 142, 1.0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0],
 [50, 57, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [57, 44, 8.0, 1.0, 0, 2.0, 0, 0, 4.0, 1.0, 1.0, 0],
 [57, 42, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0],
 [296, 115, 65.0, 0, 0, 0, 0, 69.0, 0, 0, 0, 0],
 [318, 157, 209.0, 0, 0, 0, 0, 0, 0, 8.0, 0, 0],
 [318, 156, 75.0, 0, 0, 0, 0, 65.0, 0, 1.0, 0, 1.0],
 [144, 145, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0],
 [267, 16, 0, 1.0, 0, 0, 1.0, 0, 0, 0, 0, 0],
 [50, 89, 1.0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0],


In [62]:
depends_dict[166]

'C:\\Users\\tanji\\Desktop\\SoftwareRemodularization\\raw_sourcecode\\JodaOrg-joda-time\\JodaOrg-joda-time_v2.9\\src\\main\\java\\org\\joda\\time\\tz\\Provider.java'

In [65]:
arr_a_rsf

['AgeCalculator.java',
 'DateTimeBrowser.java',
 'DateTimePerformance.java',
 'Examples.java',
 'TimeZoneTable.java',
 'Chronology.java',
 'DateMidnight.java',
 'DateTime.java',
 'DateTimeComparator.java',
 'DateTimeConstants.java',
 'DateTimeField.java',
 'DateTimeFieldType.java',
 'DateTimeUtils.java',
 'DateTimeZone.java',
 'Days.java',
 'Duration.java',
 'DurationField.java',
 'DurationFieldType.java',
 'Hours.java',
 'IllegalFieldValueException.java',
 'IllegalInstantException.java',
 'Instant.java',
 'Interval.java',
 'JodaTimePermission.java',
 'LocalDate.java',
 'LocalDateTime.java',
 'LocalTime.java',
 'Minutes.java',
 'MonthDay.java',
 'Months.java',
 'MutableDateTime.java',
 'MutableInterval.java',
 'MutablePeriod.java',
 'Partial.java',
 'Period.java',
 'PeriodType.java',
 'ReadableDateTime.java',
 'ReadableDuration.java',
 'ReadableInstant.java',
 'ReadableInterval.java',
 'ReadablePartial.java',
 'ReadablePeriod.java',
 'ReadWritableDateTime.java',
 'ReadWritableInstant.j

In [95]:
gg_arr = []
with open(r'C:\Users\tanji\Desktop\SoftwareRemodularization\MoJo_1.2.1\JodaOrg-joda-time\JodaOrg-joda-time_v2.9_hillclimbing_b.rsf', 'r') as f:
    for line in f:
        line = line.split(' ')
        gg_arr.append(int(line[-1].replace('\n','')))

In [96]:
gg_arr.sort()

In [97]:
gg_arr

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,


In [98]:
gg_arr_2 = []
with open(r'C:\Users\tanji\Desktop\SoftwareRemodularization\MoJo_1.2.1\JodaOrg-joda-time\JodaOrg-joda-time_v2.9_hillclimbing_a.rsf', 'r') as f:
    for line in f:
        line = line.split(' ')
        gg_arr_2.append(int(line[-1].replace('\n','')))

In [99]:
gg_arr_2.sort()

In [101]:
gg_arr_2

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 147,
 149,
 150,
 151,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 170,
 171,
 172,
 173,
 174,
 175,
 177,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193

In [102]:
list(set(gg_arr) - set(gg_arr_2))

[256,
 257,
 260,
 262,
 264,
 146,
 148,
 152,
 153,
 154,
 155,
 284,
 169,
 297,
 48,
 176,
 307,
 312,
 81,
 213,
 251,
 255]

In [104]:
list(set(arr_a_rsf) - set(gg_arr_2))

[]

In [106]:
list(set(arr_b_rsf) - set(arr_a_rsf))

[256,
 257,
 260,
 262,
 264,
 146,
 148,
 152,
 153,
 154,
 155,
 284,
 169,
 297,
 48,
 176,
 307,
 312,
 81,
 213,
 251,
 255]

In [107]:
list(set(arr_a_rsf) - set(arr_b_rsf))

[177]